# Lab 2 - Linear regression </br> M1 Data Science, ML1

# Introduction

In [31]:
# %%
# "IPython magic command" to sutomatically reload the imported packages after changes in a package
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import sklearn

# plt.rc("font", **{"family": "sans-serif", "sans-serif": ["Helvetica"]})
# plt.rc("text", usetex=True)

SMALL_SIZE = 16
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc("font", size=SMALL_SIZE)  # default text size
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

# display backend for matplotlib
%matplotlib inline
# %matplotlib widget
# %matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<div class="alert alert-danger" style="margin-top: 0px">


Lab report written by: Kalakech Mazen, Mchawrab Ali 2025-2026.

</div>

# Exercise 1: linear regression

Let $N, M$ be two postive integers, and consider a dataset $\mathcal{D} = \{ (y_n \mathbf{x}_n) \}_{1 \leq n \leq N}$, with targets $\mathbf{y} = (y_n)_{1 \leq n \leq N} \in \mathbb{R}^N$, and (standardized) data $\mathbf{X} = [\mathbf{x}_1, \dotsc, \mathbf{x}_N]^T \in \mathbb{R}^{N \times M}$.

This exercise is aimed at performing linear regression with `numpy` and `sklearn` on the `California house` dataset, loaded below.

For the sake of simplicity, the dataset is NOT split into a train and a test set until question 5, where the quality of the model is properly assessed.

In [22]:
# loading the dataset
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

# X = housing.data, y = housing.target
X = housing.data
y = housing.target

print(X.shape, y.shape)

print(housing.feature_names)
print(housing.DESCR)
print(y)

(20640, 8) (20640,)
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median ho

In [23]:
features_min = np.min(X, axis=0)
features_max = np.max(X, axis=0)
range_features = np.abs(features_max - features_min)

with np.printoptions(precision=2):
    print(
        "Feature ranges: \n names: {} \n dr.  : {} \n min  : {} \n max  : {}".format(
            housing.feature_names, range_features, features_min, features_max
        )
    )

Feature ranges: 
 names: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'] 
 dr.  : [1.45e+01 5.10e+01 1.41e+02 3.37e+01 3.57e+04 1.24e+03 9.41e+00 1.00e+01] 
 min  : [   0.5     1.      0.85    0.33    3.      0.69   32.54 -124.35] 
 max  : [ 1.50e+01  5.20e+01  1.42e+02  3.41e+01  3.57e+04  1.24e+03  4.20e+01
 -1.14e+02]


<div class="alert" style="margin-top: 0px">

1. Standardize the data using `sklearn`, and store the output into a variable `scaled_X`. Is it useful for this dataset? Justify your answer.

> Indication: take a look at [`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) 

</div>

In [24]:
# Looking at the Dataset, we can see that there is a big discrepency between the features ranges. This means that the cost function will be dominated by some features
import sklearn.preprocessing as pp 
scaler = pp.StandardScaler()
scaled_X = scaler.fit_transform(X) 

In [25]:
features_min = np.min(scaled_X, axis=0)
features_max = np.max(scaled_X, axis=0)
range_features = np.abs(features_max - features_min)

with np.printoptions(precision=2):
    print(
        "Feature ranges: \n names: {} \n dr.  : {} \n min  : {} \n max  : {}".format(
            housing.feature_names, range_features, features_min, features_max
        )
    )

Feature ranges: 
 names: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'] 
 dr.  : [  7.63   4.05  57.02  71.18  31.51 119.65   4.41   5.01] 
 min  : [-1.77 -2.2  -1.85 -1.61 -1.26 -0.23 -1.45 -2.39] 
 max  : [  5.86   1.86  55.16  69.57  30.25 119.42   2.96   2.63]


<div class="alert alert-warning" style="margin-top: 0px">
Answer question 1:

...

</div>

<div class="alert" style="margin-top: 0px">

2. Write ERM problem corresponding to linear least-squares regression. Recall the form of the analytic expression solution, with the definition of the elements involved.

    > Indication: type your answer in $\LaTeX$ in the cell below, or include a picture of your handwritten answer.

</div>

<div class="alert" style="margin-top: 0px">

3. 
   1. Compute the solution to the problem with `numpy` from the analytic expression, calling the result `beta_numpy`.
   
   2. Compute the solution with `sklearn`, `beta_sklearn`. 

   3. Test whether the two array are significantly different. 

> Hints: 
> - the function [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html#numpy.linalg.solve) and the class [`sklearn.linear_model.LinearRegression`](https://scikit-learn.org/stable/modules/linear_model.html) will be useful
> - the function [`numpy.allclose`](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html#numpy-allclose) can be useful to test equality between two arrays.

</div>

In [26]:
from sklearn import linear_model
# Computing X_tilda and its transpose to use the numpy solution
X_tilda = np.c_[np.ones((np.shape(scaled_X)[0],1)),scaled_X]
X_tildaT = np.transpose(X_tilda)
# Beta* = (X_tildaT * X_tilda)-1 X_tilda y
# we are solving this equation using Numpy to find Beta* 
A = np.matmul(X_tildaT, X_tilda)
b = np.matmul(X_tildaT, y)
beta_numpy = np.linalg.solve(A, b)
print(beta_numpy)
## Using sklearn

regression = linear_model.LinearRegression()
regression.fit(A,b)
beta_sklearn=regression.coef_
b=regression.intercept_
print(beta_sklearn)
#comparing the two solutions

print(np.allclose(beta_numpy,beta_sklearn))




[ 2.06855817  0.8296193   0.11875165 -0.26552688  0.30569623 -0.004503
 -0.03932627 -0.89988565 -0.870541  ]
[ 2.09471497  0.98418508  0.21359445 -0.44461931  0.46112547  0.05130131
 -0.02362868 -0.3766683  -0.36058933]
False


<div class="alert" style="margin-top: 0px">

4. Evaluate $\kappa(\widetilde{\mathbf{X}}^T\widetilde{\mathbf{X}})$, the condition number of the matrix $\widetilde{\mathbf{X}}^T\widetilde{\mathbf{X}}$ using the function [`numpy.linalg.cond`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.cond.html#numpy.linalg.cond). Is the least-squares problem considered well-conditioned? Justify your answer.

</div>

In [27]:
k = np.linalg.cond(A)
print(k)
''' A = X_tilta*X_tildaT is ill-conditioned because k == 44.4651 >> 1'''


44.46514975130451


' A = X_tilta*X_tildaT is ill-conditioned because k == 44.4651 >> 1'

<div class="alert alert-warning" style="margin-top: 0px">
Answer question 4:

...

</div>

<div class="alert" style="margin-top: 0px">

5. To properly assess the quality of the regressor, the dataset will now be split into a train and a test set. Complete the code below to:
    - split the dataset into a train and a test set
    - use `sklearn` to apply K-fold cross validation on the train set, and return the regressor associated with the best MSE criterion;
    - evaluate the performance on the test set in terms of the MSE criterion.

    > Indications: 
    > - if needed, take a look at the documentation of [`sklearn.model_selection.train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#train-test-split), and take some inspiration from [`sklearn` examples](https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-py).
    >
    > - the documentation and examples for [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) (or [`sklearn.pipeline.make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html)) and [`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) can be useful;
    >
    > - to return the regression coefficients from the pipeline, take a look its [named_steps](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.named_steps) method (if needed, see also the [stackoverflow answer](https://stackoverflow.com/questions/43856280/return-coefficients-from-pipeline-object-in-sklearn));
    >
    > - for the evaluation metrics, see [the documentation](https://scikit-learn.org/stable/modules/model_evaluation.html) and the module [sklearn.metrics](https://scikit-learn.org/stable/api/sklearn.metrics.html) 
</div> 

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# TODO: instructions below to be completed
# X_train, X_test, y_train, y_test = ...


# TODO: instructions below to be completed
# Create a linear regression model
# linear_regression = ...

# Create scaler
# scaler = ...

# TODO: uncomment once code for linear_regression and scaler variables are filled-in
# pipe = Pipeline(steps=[("scaler", scaler), ("linearregression", linear_regression)])

# Set-up cross-validation
kf = KFold(n_splits=5)

fold_id = 0
for train_index, test_index in kf.split(X_train):
    X_subtrain, X_subtest = X_train[train_index], X_train[test_index]
    y_subtrain, y_subtest = y_train[train_index], y_train[test_index]

    # train the model using the scaled train set
    # TODO: add missing instruction
    # ...

    # make predictions using the test set
    # TODO: add missing instruction, using the pipe object
    # y_pred = ...

    # TODO: compute the errors (MSE, MAE, R2), and display these below
    # display mean squared error
    print("Fold {}: MSE: {:.2f}".format(fold_id, 0))
    # display mean absolute error
    print("        MAE: {:.2f}".format(0))
    # display coefficient of determination: 1 is perfect prediction
    print("        R2 : {:.2f}".format(0))
    fold_id += 1

NameError: name 'X_train' is not defined

# Exercise 2: least-squares and matrix inversion

Let $N, M$ be two positive integers, $\mathbf{A} \in \mathbb{R}^{N \times M}$ such that $\text{rank}(\mathbf{A}) = M$, and $\mathbf{y} \in \mathbb{R}^N$. 

This exercise is aimed at comparing the accuracy and speed of 2 approaches to compute $\widehat{\boldsymbol{\beta}} = (\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A}^T\mathbf{y}$, solution to the problem 

\begin{equation*}
    \underset{\boldsymbol{\beta} \in \mathbb{R}^M}{\text{minimize}} \; \frac{1}{2} \| \mathbf{y} - \mathbf{A}\boldsymbol{\beta} \|_2^2
\end{equation*}

1. computing $\mathbf{B} = (\mathbf{A}^T\mathbf{A})^{-1}$, and then $\widehat{\boldsymbol{\beta}} = \mathbf{B}\mathbf{A}^T \mathbf{y}$.
2. solving the problem with an iterative solver, e.g., using [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html#numpy.linalg.solve).

To do so, we will use the data generated below, knowing the ground truth value $\boldsymbol{\beta}^*$ to evaluate numerical accuracy.

In [29]:
rng = np.random.default_rng(1234)

N = 200
M = 100

A = rng.standard_normal(size=(N, M))
beta_true = rng.standard_normal(size=(M,))
y = A @ beta_true

<div class="alert" style="margin-top: 0px">

1. 
   - Compute $\widehat{\boldsymbol{\beta}}$ with the two approaches mentioned above, registering the time needed to solve the problem with the 2 version. To do so, complete the code cell below.
   - Compare the time required between the two approaches

> Indication: see the functions [`np.linalg.inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html#numpy.linalg.inv) and [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html#numpy.linalg.solve)

</div>

In [59]:
import timeit
start1 = timeit.timeit()
B = np.linalg.inv(np.matmul(np.transpose(A),A))
beta_inv = np.matmul(B,np.transpose(A)).dot(y)
end1 = timeit.timeit()
algo1_time = np.abs(start1 - end1)
print(f"Time taken by algo 1: {algo1_time}")



Time taken by algo 1: 0.002269299999170471


In [ ]:
start2 = timeit.timeit()
beta_solv = np.linalg.solve(np.matmul(np.transpose(A),A),np.matmul(np.transpose(A),y))
end2 = timeit.timeit()
algo2_time = np.abs(start2 - end2)
print(f"Time taken by algo 2: {algo2_time}")

Time taken by algo 2: 0.0010371999997005332
Second approach is faster


In [ ]:
if algo1_time < algo2_time:
    print("First approach is faster")
else:
    print("Second approach is faster")
'''Result: method 2 is faster'''


Second approach is faster
45.705724235653086


<div class="alert" style="margin-top: 0px">

2. The accuracy of $\widehat{\boldsymbol{\beta}}$ can be assess with respect to the ground truth $\boldsymbol{\beta}^*$ in terms of the reconstruction signal to noise ratio (rSNR), expressed in dB, defined by

\begin{equation*}
    \text{rSNR}(\widehat{\boldsymbol{\beta}}, \boldsymbol{\beta}^*) = 10 \log_{10} \Big( \frac{\| \boldsymbol{\beta}^* \|_2^2}{\| \boldsymbol{\beta}^* - \widehat{\boldsymbol{\beta}} \|_2^2} \Big).
\end{equation*}

- Compute the rSNR for the solution `beta_inv` and `beta_solve` computed above, and indicate which of the two esimates is the most precise.
- Conclude on the best approach to adopt to solve linear systems of equations.

</div>

In [ ]:
rSNR_Algo1 = 10*np.log10(np.square(np.linalg.norm(beta_true))/np.square(np.linalg.norm(beta_true - beta_inv)))
print(f"rSNR of beta_inv = {rSNR_Algo1}")
rSNR_Algo2 = 10*np.log10(np.square(np.linalg.norm(beta_true))/np.square(np.linalg.norm(beta_true - beta_solv)))
print(f"rSNR of beta_solv = {rSNR_Algo2}")
if rSNR_Algo2 < rSNR_Algo1:
    print("first approach is more accurate")
else:
    print("second approach is more accurate")
    
'''We want to compute the ratio of the two methods execution time'''
print(f"ratio of algo2_time over algo1_time : {(algo2_time/algo1_time) * 100}")
'''We want to compute the ratio of the two methods rSNRs'''
print(f"ratio of rSNR_Algo1 over rSNR_Algo2 : {(rSNR_Algo2/rSNR_Algo1) * 100}")
'''Conclusion: The second method is almost two times (45%) faster than the first one and 99.95% accurate. 
So for larger dimensions, one may use the second method for faster execution time, but for lower dimensions, method 1 is more useful'''

rSNR of beta_inv = 296.7504037649167
rSNR of beta_solv = 296.60295904998856
first approach is more accurate
ratio of algo2_time over algo1_time : 45.705724235653086
ratio of rSNR_Algo1 over rSNR_Algo2 : 99.9503135587829


'Conclusion: the rSNR of the first method is higher than the second one by'

<div class="alert alert-warning" style="margin-top: 0px">
Answer question 2:

...

</div>

<div class="alert" style="margin-top: 0px">

3. Use `numpy` to compute the condition number $\kappa(\mathbf{A})$ of the matrix $\mathbf{A}$. What information does this value convey about the inversion problem considered?

> Indication: take a look at the documentation of the function [`numpy.linalg.cond`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.cond.html#numpy.linalg.cond).

</div>

In [70]:
k = np.linalg.cond(A)
print(k)

5.338572426355578


<div class="alert alert-warning" style="margin-top: 0px">
Answer question 3:

...

</div>

---

End of lab2.